In [1]:
import numpy as np
import json
import pandas as pd
from tqdm import trange
import argparse
from modules.gem import GeM
from utils.train_util import set_seed
from torch.utils.data import DataLoader
from datasets.dl import GeMData, GeMClass
from datasets.config import GeMConfig
import torch
import os
from tqdm import tqdm
import gc
import torch.nn.functional as F
from matplotlib import pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
from sklearn.metrics import roc_auc_score, accuracy_score
# InteractiveShell.ast_node_interactivity = "all"

In [2]:
mi = GeMConfig('cifar100')
mi.set_arch('resnet50')
model = GeM(mi)

<All keys matched successfully>


In [3]:
# pic_matrix = torch.ByteTensor(np.load("data/imageset_small.npy"))
pic_matrix = torch.ByteTensor(np.random.randint(low=0, high=255, size=(1000, 3, 64, 64)))
dataset_img = torch.LongTensor(np.random.randint(low=0, high=1000, size=(1280, 1)))
dataset_label = torch.LongTensor(np.random.randint(low=0, high=100, size=(1280, 1)))

In [4]:
train_dataset = GeMClass(pic_matrix, torch.cat([dataset_img, dataset_label], dim=-1))
train_data_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# input_label = torch.zeros((32), dtype=torch.long).to(0)
input_label = torch.randint(low=0, high=100, size=(32, 1)).reshape(-1)

In [5]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.0001)

In [6]:
# model.to(0)

In [7]:
for epoch in range(2):
    model.train()
    model.zero_grad()
    index = 0
    steps_one_epoch = len(train_data_loader)
    enum_dataloader = tqdm(train_data_loader, total=steps_one_epoch, desc="EP-{} train".format(epoch))
    loss_list = []
    for data in enum_dataloader:
    #     if index >= steps_one_epoch:
    #         break

        model_in = data[:, :-1] / 255.0
        pred = model.predict_class(model_in, 64, scale=2)
        loss = F.cross_entropy(pred, data[:, -1])
        loss_list.append(loss)

        loss.backward()
        optimizer.step()
        model.zero_grad()

        enum_dataloader.set_description("EP-{} train loss: {}".format(epoch, loss))
        enum_dataloader.refresh()
        index += 1
    
    print('epoch {} end'.format(epoch))
#     print(loss_list)

EP-0 train loss: 4.606335163116455:  22%|██████████▊                                     | 9/40 [00:24<01:24,  2.73s/it]


KeyboardInterrupt: 

In [23]:
model.eval() 
labels = []
preds = []
with torch.no_grad():
    for data in train_data_loader:
        input_data = data[:, :-1] / 255.0
        label_data = data[:, -1]
        res = model.predict_class(input_data, 64)
        maxidx = res.argmax(dim=-1)
        labels += label_data.cpu().numpy().tolist()
        preds += maxidx.cpu().numpy().tolist()

In [24]:
maxidx

tensor([88, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44,
        44, 44, 44, 44, 44, 44, 44, 44, 44, 60, 44, 44, 44, 44])

In [25]:
accuracy_score(labels, preds)

0.00703125

In [18]:
roc_auc_score(labels, preds)

0.484004884004884

In [12]:
# test dataset

pic_matrix = torch.ByteTensor(np.random.randint(low=0, high=256, size=(1000, 3, 224, 224)))
tdataset = np.random.randint(low=0, high=1000, size=(128))

In [13]:
test_dataset = GeMData(tcfg)
test_data_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [14]:
model.eval() 
preds = []
with torch.no_grad():
    for data in test_data_loader:
        input_data = data / 256.0
        print(data.size())
        input_data = input_data.to(0)
        res = model.predict(input_data, 224)
        print(res.size())

torch.Size([32, 3, 224, 224])
torch.Size([32, 512])
torch.Size([32, 3, 224, 224])
torch.Size([32, 512])
torch.Size([32, 3, 224, 224])
torch.Size([32, 512])
torch.Size([32, 3, 224, 224])
torch.Size([32, 512])


In [37]:
import math
math.sqrt(2) * 2

2.8284271247461903